In [2]:
import pandas as pd
import sbol2
import os
from rdflib import Graph


In [8]:
current_dir = os.path.abspath('')
data_path = os.path.join(current_dir, '..', 'data')
attachments_path = os.path.join(current_dir, '..', 'attachments')
pulled_attachments_path = os.path.join(current_dir, '..', 'pulled_attachments')
sbol_path = os.path.join(current_dir, '..', 'sbol_data')
downloaded_sbol_path = os.path.join(current_dir, '..', 'downloaded_sbol')
original_data_path = os.path.join(data_path, 'original_data')
scripts_path = os.path.join(current_dir, 'scripts')
model_data_path = os.path.join(data_path, 'processed_data', 'replicated_models')
model_output_path = os.path.join('..', 'model_outputs')

In [10]:
g = Graph()
g.parse(os.path.join(sbol_path, "ecolipromoter_one.xml"), format="xml") 
g.serialize(destination="ecolipromoters_one.nt", format="nt")

print("Done: RDF/XML has been converted to N-Triples (output.nt)")

Done: RDF/XML has been converted to N-Triples (output.nt)


In [1]:
import autordf2gml

#to run content-based transformation
a = autordf2gml.content_feature("config.ini") 

c:\Users\Sai\Documents\GitHub\SBOLtrainer\autordf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###### AutoRDF2GML (content-based): START! ######
## start_time=1752627459.4574063
## Loading the config file: config.ini
## Path exists: .
## Path exists: .
## Configs: input:ecolipromoters_one.nt / nld_class='ModuleDefinition' / embedding_model='allenai/scibert_scivocab_uncased' / output:save_path_mapping='.' .
## Loading the RDF dump from: file_path='ecolipromoters_one.nt', format: nt
## RDF dump file loaded. The RDF graph contains 189 triples.
## Transformation started! Querying the graph...
## Automatic feature selection...
## NLD column: pivoted_df_ModuleDefinition


KeyError: 'pivoted_df_work'